In [16]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import sys
sys.path.append('../')

In [18]:
import os
import h2o
import pandas as pd

from h2o.automl import H2OAutoML
from utilities import save_model as save

home_dir = os.path.join(os.path.expanduser('~'), 'Documents/TDT4173-ML')
train_data_path = os.path.join(home_dir, 'data_processed')
# target_data_path = os.path.join(home_dir, 'data_processed')

In [19]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "16.0.1" 2021-04-20; OpenJDK Runtime Environment (build 16.0.1+9-Ubuntu-120.04); OpenJDK 64-Bit Server VM (build 16.0.1+9-Ubuntu-120.04, mixed mode, sharing)
  Starting server from /home/stinky/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprcaxl387
  JVM stdout: /tmp/tmprcaxl387/h2o_stinky_started_from_python.out
  JVM stderr: /tmp/tmprcaxl387/h2o_stinky_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_stinky_tjonwt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.932 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [20]:
train = pd.read_csv(os.path.join(train_data_path, 'imputed_5n_mean_train.csv'))
# target = pd.read_csv(os.path.join(target_data_path, 'target.csv'))

train.set_index('date_forecast', inplace=True)
# target.set_index('time', inplace=True)

In [21]:
X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [22]:
autoML = H2OAutoML(max_models=30, seed=10)
autoML.train(x=X, y=y, training_frame=X_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/30
# GBM base models (used / total),1/10
# XGBoost base models (used / total),7/10
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [23]:
best_model = autoML.leader
save.save_model_to_local_disk(best_model, '03E_h2o_mean_imputed_5n')